In [21]:
import joblib
import numpy as np
import pandas as pd
# Import packages
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Sklearn Packages
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier

# Sklearn Evaluation Metrics
from sklearn import metrics
from sklearn.metrics import mean_squared_error, precision_score, confusion_matrix, accuracy_score

# Visualizes all the columns
pd.set_option('display.max_columns', None)

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

In [39]:
array = joblib.load('Dataset.joblib')
len(array)


916

In [24]:
df = pd.DataFrame(array)

In [25]:
max_length = df.shape[1] - 2

In [26]:
df['EC'] = df[1].apply(lambda x: 1 if x == 'CONFIRMED' else 1 if x == 'CANDIDATE' else 0 )

In [27]:
features = df.drop(columns=[0,1,'EC'])
target = df.EC

In [28]:
medians=features.median(axis=1)

In [29]:
df_dict = features.to_dict(orient='index')
    

In [30]:
for x in range(0,len(df_dict)):
    for y in range(2,max_length+2):
        df_dict[x][y] = df_dict[x][y] /medians[x] -1
    

In [31]:

for x in range(0,len(df_dict)):
    for y in range(2,max_length+2):
        if str(df_dict[x][y]) == "nan":
            df_dict[x][y] = -999
        else:
            df_dict[x][y] = float(df_dict[x][y])


In [32]:
features = pd.DataFrame.from_dict(df_dict,orient='index')

In [33]:
X_train, X_test, y_train, y_test = train_test_split(features, target, random_state=1, test_size=.25)

In [34]:
# Evaluation function

def evaluation(y_true, y_pred):
    
# Print Accuracy, Recall, F1 Score, and Precision metrics.
    print('Evaluation Metrics:')
    print('Accuracy: ' + str(metrics.accuracy_score(y_test, y_pred)))
    print('Recall: ' + str(metrics.recall_score(y_test, y_pred)))
    print('F1 Score: ' + str(metrics.f1_score(y_test, y_pred)))
    print('Precision: ' + str(metrics.precision_score(y_test, y_pred)))
    
# Print Confusion Matrix
    print('\nConfusion Matrix:')
    print(' TN,  FP, FN, TP')
    print(confusion_matrix(y_true, y_pred).ravel())
    
# Function Prints best parameters for GridSearchCV
def print_results(results):
    print('Best Parameters: {}\n'.format(results.best_params_))

In [35]:
tree = DecisionTreeClassifier()

# Fitting Model to the train set
tree.fit(X_train, y_train)

# Predicting on the test set
y_pred = tree.predict(X_test)

# Evaluating model
evaluation(y_test, y_pred)

Evaluation Metrics:
Accuracy: 0.9344978165938864
Recall: 0.9385474860335196
F1 Score: 0.9572649572649573
Precision: 0.9767441860465116

Confusion Matrix:
 TN,  FP, FN, TP
[ 46   4  11 168]


In [36]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from tqdm.notebook import tqdm
from time import sleep
parameter_n_estimators = [500]
for i in tqdm(parameter_n_estimators):
    # Instantiate model
    forest = RandomForestClassifier(n_estimators=i, criterion='gini')
    # Fitting Model to the train set
    forest.fit(X_train, y_train)
    # Predicting on the test set
    y_pred = forest.predict(X_test)

    # Evaluating model
    evaluation(y_test, y_pred)
    print('Tree: %s ' % (i))

  0%|          | 0/1 [00:00<?, ?it/s]

Evaluation Metrics:
Accuracy: 0.9344978165938864
Recall: 0.9441340782122905
F1 Score: 0.9575070821529744
Precision: 0.9712643678160919

Confusion Matrix:
 TN,  FP, FN, TP
[ 45   5  10 169]
Tree: 500 


In [37]:
knn = KNeighborsClassifier(leaf_size=8, metric='manhattan',weights='uniform')

# Fitting Model to the train set
knn.fit(X_train, y_train)

# Predicting on the test set
y_pred = knn.predict(X_test)

# Evaluating model
evaluation(y_test, y_pred)

Evaluation Metrics:
Accuracy: 0.834061135371179
Recall: 0.8938547486033519
F1 Score: 0.8938547486033519
Precision: 0.8938547486033519

Confusion Matrix:
 TN,  FP, FN, TP
[ 31  19  19 160]


In [16]:
import lightkurve as lk

T_name = 'TIC 145241359'
search_result = lk.search_lightcurve(T_name)
data = []
for x in range(0,17):
    try:
        lc = search_result[x].download() 
        y = lc.flux
        for i in range(1,len(y),10):
            try:
                data.append(float(y[i].value))
            except:
                pass
    except :
        pass

arr2 = pd.DataFrame(data)
medians= arr2.median(axis=0)
arr3 =[]
for x in range(0,max_length):
    try:
        arr3.append((arr2[0][x] / medians)-1)
    except:
        arr3.append(-999)

for x in range(0,max_length):
    temp = str(arr3[x])
    if temp == 'nan' :
        arr3[x] = -999

0    1.000336
dtype: float64


In [43]:
test1 = forest.predict([arr3])

In [44]:
print(test1)

[1]
